<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/topoflinecondensation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of line condensation in gas pipelines
In the follwing section we will demonstrate calculation related to top of line condensation in a pipeline where MEG is used as hydrate or corrosion inhibitor.

Litterature:

https://iopscience.iop.org/article/10.1088/1757-899X/1201/1/012082/pdf 

In [1]:
%%capture
#We start by installing the neqsim Python package
!pip install neqsim
import neqsim
from neqsim.thermo import fluid, TPflash, printFrame

#Saturate the gas with a MEG/water solution
Select the gas component and set wt% MEG in the bulk glycol solution. For low pressures (<10 bara) the type of gas will have little effect on results.

In [2]:
wtPercentageMEG = 60.0 #@param {type:"number"}
gascomponent = 'nitrogen' #@param ['methane', 'nitrogen', 'CO2']

In [3]:
fluid1 = fluid('cpa')
fluid1.addComponent(gascomponent, 5.0, 'kg/hr')
fluid1.addComponent("MEG", wtPercentageMEG, 'kg/hr')
fluid1.addComponent("water", 100.0- wtPercentageMEG, 'kg/hr')
fluid1.setMixingRule(10)

We set the temperature and pressure and run a TPflash to calculate the composition of the gas and liquid. We then display the resulting fluid composition.

In [4]:
temperature_C = 50.0 #@param {type:"number"}
pressure_bara = 1.01325 #@param {type:"number"}

TPflash(fluid1)

printFrame(fluid1)

| 0                  | 1          | 2                    | 3                    | 4   | 5   | 6               |
|:-------------------|:-----------|:---------------------|:---------------------|:----|:----|:----------------|
|                    | total      | gas                  | aqueous              |     |     |                 |
| nitrogen           | 5.30335E-2 | 9.79246E-1           | 2.36601E-5           |     |     | [mole fraction] |
| MEG                | 2.87226E-1 | 3.5611E-5            | 3.03663E-1           |     |     | [mole fraction] |
| water              | 6.5974E-1  | 2.07179E-2           | 6.96314E-1           |     |     | [mole fraction] |
|                    |            |                      |                      |     |     |                 |
| Density            |            | 1.13679E0            | 1.05106E3            |     |     | [kg/m^3]        |
| PhaseFraction      |            | 5.41346E-2           | 9.45865E-1           |     |     | [mole frac

# Calculation of activity coefficients and water and MEG in gas
The equilibrium can be represented by the equation using vapour pressure of pure components and activity coefficients. 

$y_{water}=\frac{x_{water}\times\gamma_{water}*P_{water}}{\varphi_{water}\times P_{total}}$

and

$y_{MEG}=\frac{x_{water}\times\gamma_{MEG}*P_{MEG}}{\varphi_{MEG}\times P_{total}}$

Vapour pressure of MEG can be calculated from the Antonie equation:

https://webbook.nist.gov/cgi/cbook.cgi?ID=C107211&Mask=4

and water:

https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Units=SI&Mask=4#Thermo-Phase


At low pressures fugacity coefficients in will be close to one ($\varphi =1) $. In the following calculations the CPA-EoS in NeqSim are used for the calculations of the fluid phase equilibrium.



In [5]:
waterInGas = fluid1.getPhase('gas').getComponent('water').getx()*1e2
MEGinGas = fluid1.getPhase('gas').getComponent('MEG').getx()*1e2

print('saturated gas compostion')
print('water in gas ', waterInGas, ' MEG in gas', MEGinGas,' [mole%] ')

water_activity_coefficient = fluid1.getPhase('aqueous').getActivityCoefficient(2)
MEG_activity_coefficient = fluid1.getPhase('aqueous').getActivityCoefficient(1)
print('activity coefficients')
print('water ', water_activity_coefficient, ' MEG ', MEG_activity_coefficient)

saturated gas compostion
water in gas  2.071794132617124  MEG in gas 0.0035610959737163743  [mole%] 
activity coefficients
water  0.9424089599110065  MEG  1.0038140122521804


#Calculation of condensed gas composition
The composition of the liquid after condensing the gas on a cold surface can be calulated by running a TPflash calculation on the saturated gas phase. A liquid phase will typically form if the temperature is reduced.

In [ ]:
coldSurfaceTemperature = 20.0 #@param {type:"number"}
coldSurfacePressure = 1.01325 #@param {type:"number"}

gasFluid = fluid1.phaseToSystem('gas')
gasFluid.setTemperature(coldSurfaceTemperature, 'C')
gasFluid.setPressure(coldSurfacePressure, 'bara')
TPflash(gasFluid)
printFrame(gasFluid)

| 0                  | 1          | 2                    | 3                    | 4   | 5   | 6               |
|:-------------------|:-----------|:---------------------|:---------------------|:----|:----|:----------------|
|                    | total      | gas                  | aqueous              |     |     |                 |
| nitrogen           | 9.79246E-1 | 9.79511E-1           | 1.50126E-5           |     |     | [mole fraction] |
| MEG                | 3.5611E-5  | 6.54532E-6           | 1.07516E-1           |     |     | [mole fraction] |
| water              | 2.07179E-2 | 2.04822E-2           | 8.92469E-1           |     |     | [mole fraction] |
|                    |            |                      |                      |     |     |                 |
| Density            |            | 1.1563E0             | 1.02174E3            |     |     | [kg/m^3]        |
| PhaseFraction      |            | 9.9973E-1            | 2.70354E-4           |     |     | [mole frac

The composition of the condensed phase is

In [ ]:
wtPrecentMEG = gasFluid.getPhase('aqueous').getWtFrac('MEG')*100
wtPrecentwater = gasFluid.getPhase('aqueous').getWtFrac('water')*100
print('composition of condensed phase')
print('water ', wtPrecentwater, ' wt%')
print('MEG ', wtPrecentMEG, ' wt%')

composition of condensed phase
water  70.66656958212609  wt%
MEG  29.331581956200054  wt%
